# Exercise 11: Handling missing values using scikit-learn and pandas in preparation for statistical data analysis using SciPy

In today's class you will work through this exercise to perform exploratory data analysis to compare the GPP and SIF data. Specifically, we will explore how we deal with missing values before we perform a statistical analysis. This kind of exploratory data analysis is a crucial first step in analyzing your data.  

We will be using the GPP and SIF datasets contained in netcdf files that we have been using in previous classes.

Follow the instructions and type your answers in the code boxes with "# type your command here". There are also some questions to answer in the text boxes. A second jupyter notebook with the answers will be provided after the deadline.

Complete the whole assignment in this Jupyter Notebook and **upload it to Canvas by 1pm Monday 21st October**.

### 1. Loading GPP and SIF data

As you have done in previous in-class exercises, firs, download the GPP (orch_gpp_monthly_2007-2011.nc) and SIF (gome2_sif_monthly_2007-2011.nc) files and set the path to where the files are stored in our script. 

Then you need to import the netcdf library, open both the GPP and SIF netcdf files, and read in the data. 

You may need to query the variable keys to check the name of the SIF variable. Follow the instructions you have learned in previous classes to write the code you in the box below and run the code. Note! You will not just be able to use f=nc.Dataset() as we now need to open two files, therefore you will need to call them different names.

In [1]:
# type your commands here

In [2]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt

gpp_file = '../data/gpp_sif/orch_gpp_monthly_2007-2011.nc'
sif_file = '../data/gpp_sif/gome2_sif_monthly_2007-2011.nc'

gpp_f=nc.Dataset(gpp_file,'r')
sif_f=nc.Dataset(sif_file,'r')

gpp_data = gpp_f.variables['GPP'][:]
sif_data = sif_f.variables['SIF'][:]

### 2. Exploratory comparison of GPP and SIF - the issue of missing values

As I mentioned in the previous in-class exercises, SIF should be a good proxy of GPP. If you compare the var info for GPP to SIF using the commands we learned in previous exercises you will see that the units of GPP are not the same as for SIF - this is why we see different ranges of data in the maps we plotted in Exercise 9. Therefore, we cannot directly compare the absolute values of GPP and SIF. When we plotted the maps we could visually compare their spatial distributions - but it is only easy to see large scale patterns. To really compare the data we would need to normalize or scale both datasets. 

However, we are not interested in comparing the absolute values of GPP and SIF as the values are not the same. Instead, we are mostly interested in comparing the temporal patterns - i.e. the correlation between the two datasets. We don't need to scale the data for this. We can visualize how well correlated the two datasets are using a scatter plot. We have plotted scatter plots many times. Type your code to plot the scatter plot between GPP and SIF in the box below:

In [3]:
# type your commands here

In [4]:
plt.scatter(gpp_data, sif_data)

NameError: name 'plt' is not defined

However, there is one more step. Given the two datasets have different ranges, if you simply plot a scatter plot you will likely not have seen any positive correlation between the datasets. 

If GPP is on the x-axis and SIF is on the y-axis in your scatter plot above, you will likely have seen a vertical thin spread of points. This is because the default x-axis range is much bigger than the range of the GPP numbers. 

We need to re-scale the axes. Re-plot your scatter plot in the box below and use   
**plt.xlims((xmin,xmax))**  
and  
**plt.ylims((ymin,ymax))**  
to re-scale the axes. 

You can estimate the min or max for each dataset or use numpy commands to do so. Also, add labels to your axes.

In [ ]:
# type your commands here

In [ ]:
print (np.min(gpp_data), np.max(gpp_data))
plt.scatter(gpp_data, sif_data)
plt.xlim((0,0.0004)) #--> try this now - see introduction to data visualization in python
plt.ylim((0,6))
plt.xlabel('GPP')
plt.ylabel('SIF')

We should now see much more of a spread in the GPP data and more of a relationship between GPP and SIF. 

- *From this scatter plot, do you think GPP and SIF are positively correlated?*

In the scatter plot above we have included all the terrestrial grid points. It is not that surprising that a really clear and well-defined relationship does not emerge. 

While we're exploring our data, we may as well look at a simple timeseries plot of both GPP and SIF from one (or several) grid points to see if it looks like there is temporal correlation between the two variables. We'll chose a grid point over Indiana for our comparison (row 26 and column 48). Therefore, our GPP and SIF indices will be [:,26,48]. 

First, we need to scale both the GPP and SIF data to their minimum and maximum values to remove the large discrepancy in values due to their different unit ranges. 

We do this using a scikit-learn function called minmax_scale which scales all the data to a specified range (default to between 0 and 1). Scikit-learn is a machine learning library for python (https://scikit-learn.org/). We will learn more about this library in the last part of the semester. 

To scale our data using minmax_scale we first import this specific function from the scikit-learn library and then we use it with the following commands:

**from sklearn.preprocessing import minmax_scale**

**gpp_ind_scaled = minmax_scale(gpp_data[:,26,48])**

Type these commands to create a scaled GPP and SIF array for the Indiana pixel in the box below, then print the gpp_ind_scaled array and its shape:

In [ ]:
# type your commands here

In [ ]:
from sklearn.preprocessing import minmax_scale

gpp_ind_scaled = minmax_scale(gpp_data[:,26,48])

sif_ind_scaled = minmax_scale(sif_data[:,26,48])

print (gpp_ind_scaled, np.shape(gpp_ind_scaled))

Now we can plot a simple timeseries For this we simply use the plt.plot() command. Create a plt.figure() as above and then plot two time series *on the same plot* using plt.plot - one for GPP and one for SIF for this Indiana pixel. Use the argument c='blue' for GPP and c='red' for SIF.

In [ ]:
# type your commands here

In [ ]:
plt.figure()
plt.plot(gpp_ind_scaled, c='blue')
plt.plot(sif_ind_scaled, c='red')

- *The SIF plot looks strange...why is this?*

We see two sharp spikes instead of a continous timeseries. This is not normal. It likely indicates the presence of missing values in the SIF timeseries. This is not unlikely, as SIF are observations. We can check this by printing out the initial SIF data for this pixel in the box below

In [ ]:
# type your commands here

In [ ]:
print (sif_data[:,26,48])

Indeed, there are two missing data points that have been masked (we can see the -- in two places). It is always tricky dealing with missing numbers! How do we deal with this? We can fill these points with NaNs using the numpy masked array filled function that we have come across before:

**np.ma.filled(sif_data[:,26,48], np.nan)**

However, the sklearn scaling and transform methods unfortunately don't deal with NaNs very well. The minmax_scale method throws an error if you have NaNs in your array. 

There are other more sophisticated scikit learn functions like the MinMaxScaler() function that you could use (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler) but as it says on the manual page it will disregard NaNs when creating the fit() of the scaler, but include them again when it uses the transform() method to scale the data.

What else can we do? The calculation to scale between a min and a max is relatively simple - the equation is:

*scaled_data = data - min(data) / max(data) - min(data)*

We can use numpy functions np.nanmin and np.nanmax to find the minimum and maximum without including the NaNs, e.g. for the SIF data:

**sif_ind_scaled = ( sif_data[:,26,48] - np.nanmin(sif_data[:,26,48]) ) / ( np.nanmax(sif_data[:,26,48]) - np.nanmin(sif_data[:,26,48]) )**

Note this also works even if you have not filled masked array with NaNs where the missing values are - it still treats the masked values as the NaNs

Try all these commands in the box below and print the result to see what we get:

In [ ]:
# type your commands here

In [ ]:
sif_data[:,26,48] = np.ma.filled(sif_data[:,26,48], np.nan)
sif_ind_scaled = ( sif_data[:,26,48] - np.nanmin(sif_data[:,26,48]) ) / ( np.nanmax(sif_data[:,26,48]) - np.nanmin(sif_data[:,26,48]) )
print (sif_ind_scaled)

The data are now scaled but we have kept the missing values out of the calculation and masked.

Now we can repeat our plt.plot() commands for both GPP and SIF below and see if we have a good correlation.

In [ ]:
# type your commands here

In [ ]:
plt.figure()
plt.plot(gpp_ind_scaled, c='blue')
plt.plot(sif_ind_scaled, c='red')

Indeed, it looks like we have a great correlation between SIF and GPP! You could also plot a scatter plot between the GPP and SIF at this point and you would see a better correlation than when all the terrestrial points are included.

*What are some other ways we can deal with missing values?*

Pandas has neat and elegant methods for dealing with NaNs. We can import pandas, save SIF to a dataframe with masked values filled as NaNs like this:

**import pandas as pd**

**sif_df = pd.DataFrame({'SIF': np.ma.filled(sif_data[:,26,48], np.nan)})**

As you may remember from the DataCamp tutorial on Dictionaries and Pandas, this is a quick way of creating a pandas dataframe using {} to create a dictionary with a key of "SIF" and the "value" is the array of data. The key "SIF" then becomes the column header for the SIF data column. We then feed this dictionary into pd.DataFrame() to create the data frame.

One pandas method we can use to retrieve the indices of our data that are NaNs using the method "isnull()".

**null_index = sif_df['SIF'].isnull()**

Then we can repeat our sklearn minmax_scale() method using the sif_df with null indices removed, like this:

**sif_ind_scaled = minmax_scale(sif_df.loc[~null_index, ['SIF']])**  
Note! Other versions of pandas might require: **sif_ind_scaled = minmax_scale(sif_df.loc[~null_index, ['SIF']])** i.e. with no [ ] around 'SIF'.

The ~null_index in the [ ] using the pandas method df.loc means "take all indices *except* these indices (specified in null_index)".

Type these commands in the box below and print out the result of each line.

In [ ]:
# type your commands here

In [ ]:
import pandas as pd

sif_df = pd.DataFrame({'SIF': np.ma.filled(sif_data[:,26,48], np.nan)})
null_index = sif_df['SIF'].isnull()
sif_ind_scaled = minmax_scale(sif_df.loc[~null_index, ['SIF']]) # - minmax_scale(sif_df.loc[~null_index, 'SIF']) should also work depending on versions.

print (sif_df)
print (null_index)
print (sif_ind_scaled)

Again, repeat your plotting code from above to see if you get the same result:

In [ ]:
# type your commands here

In [ ]:
plt.figure()
plt.plot(gpp_ind_scaled, c='blue')
plt.plot(sif_ind_scaled, c='red')
plt.legend()

- *Look carefully, is the plot the same as before? If not, how are they different and why is this the case?* Write your answer below:

ANSWER: This is because the missing values in the SIF data have been removed.

[Hint: print the length of the gpp_ind_scaled and the sif_ind_scaled - why are they not the same?]

In [ ]:
# type your commands here

In [ ]:
print (len(gpp_ind_scaled), len(sif_ind_scaled))

The solution to this is that although all the values of GPP are present, because of two of the SIF values are missing we have to remove the equivalent values of GPP if we want to perform an actual statistical analysis that will compare point by point.

As we have seen, pandas has useful methods to provide an elegant solution to dealing with NaNs. If we create a pandas dataframe for GPP we can use the same null_index indices that show us where the missing SIF points are to also remove the equivalent GPP pixels. Can you think how to do that? Create a pandas dataframe for the GPP data in the box below, re-scale the GPP data by excluding the **SIF** missing data points, then re-plot the graph one more time.

In [ ]:
# type your commands here

In [ ]:
gpp_df = pd.DataFrame({'GPP': np.ma.filled(gpp_data[:,26,48], np.nan)})
gpp_ind_scaled = minmax_scale(gpp_df.loc[~null_index, ['GPP']])
plt.figure()
plt.plot(gpp_ind_scaled, c='blue')
plt.plot(sif_ind_scaled, c='red')

Hooray, the curves now line up better again with two points remove that correspond to the indices of the missing SIF data. Be sure you understand the differences between these three red and blue timeseries plots.

Another way to deal with missing data is to "gap-fill". For example, the SIF data only had two missing points. It would have been easy to simply fill these points by taking an average of the two adjacent points before and after. We learned how to do this in the DataCamp pandas tutorial. However, if you have large chunks of data missing then this can be a trickier issue. 

*Note that if we were actually plotting a timeseries of data to show in a publication (with a proper date on the x-axis) we would not want to simply remove those missing data points as we have done in that final graph. For plotting time series we would show the missing data points by filling them with np.nans (that matplotlib would ignore). But if we want to perform statistical analyses, and you don't want to gap-fill the data, the pandas methods above can remove these points before we calculate a correlation.*

### 3. Quick statistical analysis of the relationship between GPP and SIF using the SciPy library

We can use the SciPy (the scientific computing tools for Python package - https://www.scipy.org) statistical functions to calculate the pearson correlation coefficient (https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html). We will learn more about Scipy in the last part of the semester.

You will need to import the scipy pearsonr package in the box below using the code:

**from scipy.stats import pearsonr**

Then calculate the pearson correlation coefficient between the scaled GPP and SIF data for the Indiana pixel using the following:

**correlation, pvalue = pearsonr(gpp_ind_scaled, sif_ind_scaled)**

Note that the pearsonr method gives both the correlation and its associated pvalue. Right now we're only interested in the correlation.

Type your code in the box below and print the correlation. Is it what you expected based on your plots above?

In [ ]:
# type your commands here

In [ ]:
from scipy.stats import pearsonr
correlation, pvalue = pearsonr(gpp_ind_scaled, sif_ind_scaled)
print (correlation)

There are many more useful statistical analyses we can do with scipy that we won't cover now. A similar SciPy method for calculating the correlation and p-value, while also getting the slope and intercept of the linear regression, is scipy.stats.linregress: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html

Unfortunately, most of these SciPy methods only deal with 1d arrays; therefore, if we want to compute the corelations for the whole globe we would have to loop over all the land pixels. We will think about this in the last part of the semester.

**That's it, you're done for today! Save this Jupyter Notebook and upload it to Canvas by 1pm Monday 21st October.**